In [1]:
###

import pandas as pd

outfile = "dogbreeds_merged.tsv"
template_url="https://docs.google.com/spreadsheets/d/e/2PACX-1vSuwLXikgq08frK7d8yFSdWTS8P1erx5bS_QiLdHhfKV4ulJlRrqkVaVhC7b3O6Z8ixrvJgoCBy8YLq/pub?gid=1655315858&single=true&output=tsv"
term_to_obsolete = "VBO:0200238"
term_to_merge_into = "VBO:0200239"
issue = 123

In [2]:
def merge(df, term_to_obsolete, term_to_merge_into, issue=1):
    df_term_to_merge_into = df[df["vbo_id"]==term_to_merge_into].copy()
    df_term_to_obsolete = df[df["vbo_id"]==term_to_obsolete].copy()
    df_term_to_obsolete2 = df_term_to_obsolete.copy()
    
    term_to_merge_into_label = df_term_to_merge_into["term_label"].values[0]
    term_to_obsolete_label = df_term_to_obsolete["term_label"].values[0]
    
    list_of_columns_to_keep = ["term_label", "vbo_id", "contributor"]
    for column in df_term_to_obsolete.columns:
        if column not in list_of_columns_to_keep:
            df_term_to_obsolete[column] = ''
    
    df_term_to_obsolete['term_label'] =  "obsolete " + df_term_to_obsolete['term_label']
    df_term_to_obsolete['obsoletion_type'] =  "owl:Class"
    df_term_to_obsolete['obsolete'] =  "true"
    df_term_to_obsolete['contributors'] =  ""
    df_term_to_obsolete['replacement_term'] =  term_to_merge_into
    df_term_to_obsolete['replacement_label'] = term_to_merge_into_label
    df_term_to_obsolete['obsolescence_reason'] = "terms merged"
    df_term_to_obsolete['GH_issue'] = "https://github.com/monarch-initiative/vertebrate-breed-ontology/issues/{issue}".format(issue=issue)
    
    # Update duplicated
    df_term_to_obsolete2['vbo_id'] =  term_to_merge_into
    df_term_to_obsolete2['term_label'] =  ''
    df_term_to_obsolete2['synonym_label_from_merged_term'] =  term_to_obsolete_label
    df_term_to_obsolete2['source_for_merged_term'] =  term_to_obsolete
    df_term_to_obsolete2['synonym_type_most_common_name'] =  ''
    df_term_to_obsolete2['GH_issue'] = "https://github.com/monarch-initiative/vertebrate-breed-ontology/issues/{issue}".format(issue=issue)    

    return pd.concat([df_term_to_obsolete, df_term_to_obsolete2, df_term_to_merge_into])


def obsolete_and_write(template_url, term_to_obsolete, term_to_merge_into, issue, outfile):
    df=pd.read_csv(template_url,sep="\t")
    out = merge(df, term_to_obsolete, term_to_merge_into,issue=issue)
    out.to_csv(outfile,sep="\t",index=False)


In [4]:
obsolete_and_write(
        template_url=template_url,
        term_to_obsolete = "VBO:0200238",
        term_to_merge_into = "VBO:0200239",
        issue = 123,
        outfile = outfile
)
